Keras Implementation of MLP Network on MNIST Dataset 

[Kaggle](https://www.kaggle.com/u6yuvi/mnist-in-keras?scriptVersionId=10035046)

# Imports

In [57]:
import os
from pathlib import Path
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
PATH=Path("../input/")
print(os.listdir("../input/"))

['train.csv', 'sample_submission.csv', 'test.csv']


# Load Data

In [58]:
train=pd.read_csv(PATH/'train.csv')
test=pd.read_csv(PATH/'test.csv')
train.shape,test.shape

((42000, 785), (28000, 784))

# Extract Input and Target Variable

In [59]:
x=train.drop("label",axis=1)
y=np.array(train['label'])
x.shape,y.shape

((42000, 784), (42000,))

# Train Test Split

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size=0.2,random_state=123)
print(x_train.shape,x_valid.shape)
x_train = x_train.values.reshape(33600, 784)
x_valid = x_valid.values.reshape(8400, 784)
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_train /= 255
x_valid /= 255
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'valid samples')

(33600, 784) (8400, 784)
33600 train samples
8400 valid samples


# Convert class vectors to binary class matrices

In [62]:
y_train.shape,y_train[:2]

((33600,), array([2, 9]))

In [63]:
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)
print(y_train.shape,y_valid.shape)

(33600, 10) (8400, 10)


# Model Architecture

In [64]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               200960    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                650       
Total para

In [65]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])

In [66]:
epochs=5
batch_size=64
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid, y_valid))

Train on 33600 samples, validate on 8400 samples
Epoch 1/5
33600/33600 [==============================] - 4s 130us/step - loss: 0.4510 - acc: 0.8685 - val_loss: 0.2539 - val_acc: 0.9343
Epoch 2/5
33600/33600 [==============================] - 4s 116us/step - loss: 0.3325 - acc: 0.9151 - val_loss: 0.2516 - val_acc: 0.9357
Epoch 3/5
33600/33600 [==============================] - 4s 116us/step - loss: 0.2982 - acc: 0.9242 - val_loss: 0.1931 - val_acc: 0.9508
Epoch 4/5
33600/33600 [==============================] - 4s 115us/step - loss: 0.2895 - acc: 0.9277 - val_loss: 0.2211 - val_acc: 0.9506
Epoch 5/5
33600/33600 [==============================] - 4s 115us/step - loss: 0.2965 - acc: 0.9298 - val_loss: 0.2089 - val_acc: 0.9539


In [67]:
score = model.evaluate(x_valid, y_valid, verbose=0)
print('Valid loss:', score[0])
print('Valid accuracy:', score[1])

Valid loss: 0.20888403075259357
Valid accuracy: 0.9539285714285715


In [68]:
#from pathlib import Path
#import simplejson
#serialize model to JSON
#filepath_json=Path('../input/')
#model_json = model.to_json()
#with open(filepath_json/"mnist_keras.json", "w") as json_file:
 #   json_file.write(simplejson.dumps(simplejson.loads(model_json), indent=4))

In [69]:
model.save_weights("mnist_keras.h5")

# Load Test Data

In [70]:
test = pd.read_csv("../input/test.csv")
print(test.shape)

(28000, 784)


# Preprocessing

In [71]:
x_test=test.loc[:,test.columns != "label"]
x_test = x_test.astype('float32')
x_test /= 255
print(x_test.shape[0], 'test samples')

28000 test samples


In [72]:
score = model.predict(x_test, verbose=0)
print(score.shape)

(28000, 10)


# Take a look

In [73]:
np.argmax(score,axis=1)[:4],np.argmax(score,axis=1).shape

(array([2, 0, 9, 9]), (28000,))

# Generate Predictions

In [74]:
predictions=np.argmax(score,axis=1)
print("Prediction shape",predictions.shape)

Prediction shape (28000,)


# Create Submission File

In [75]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("my_submissions_keras.csv", index=False, header=True)